In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path


In [2]:
def get_ground_truth(args):
    (collectionName, phoneName), df = args
    
    path = root_dir / f"train/{collectionName}/{phoneName}/ground_truth.csv"
    target_df = pd.read_csv(path)
    output_df = pd.DataFrame()
    # merge derived and target by 'millisSinceGpsEpoch'
    for epoch, epoch_df in df.groupby('millisSinceGpsEpoch'):
        idx = (target_df['millisSinceGpsEpoch'] - epoch).abs().argmin()
        epoch_diff = epoch - target_df.loc[idx, 'millisSinceGpsEpoch']
        epoch_df['epoch_diff'] = epoch_diff
        epoch_df['target_latDeg'] = target_df.loc[idx, 'latDeg']
        epoch_df['target_lngDeg'] = target_df.loc[idx, 'lngDeg']
        output_df = pd.concat([output_df, epoch_df]).reset_index(drop=True)    
    return output_df

In [3]:
# metric
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [4]:
import simdkalman

# define kf model
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

In [5]:
def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in tqdm(unique_paths):
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

In [6]:
def linear_interpolation(input_df, speed_thr=50):
    dfs = pd.DataFrame()
    for (collectionName, phoneName), df in input_df.groupby(['collectionName','phoneName']):
        df = df.sort_values('millisSinceGpsEpoch')
        df['delta'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg'].shift(1), df['lngDeg'].shift(1))
        df['time_delta'] = df['millisSinceGpsEpoch'] - df['millisSinceGpsEpoch'].shift(1)
        df['delta'].fillna(0, inplace=True)
        df['time_delta'].fillna(0, inplace=True)
        df['speed'] = df['delta'] / (df['time_delta']/1000)  # m/s
        df['speed'].fillna(0, inplace=True)

        # 一度欠損値にする
        df.loc[speed_thr<df['speed'], ['latDeg', 'lngDeg']] = np.nan
        df['dummy_datetime'] = pd.to_datetime(df['millisSinceGpsEpoch'])
        df = df.set_index('dummy_datetime')

        # 時間に合わせて線形補間
        df = df.interpolate(method='time').reset_index(drop=True)
        dfs = pd.concat([dfs, df]).reset_index(drop=True)
    return dfs

In [7]:
# config
EXP_NAME = str(Path().resolve()).split('/')[-1]

In [8]:
EXP_NAME

'exp010'

In [9]:
# waypointを補正したdataset
root_dir = Path('../../input/google-smartphone-decimeter-challenge/')

## LI -> KF  

In [10]:
import multiprocessing
train_df = pd.read_csv(root_dir / "baseline_locations_train.csv")
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    gr = train_df.groupby(['collectionName','phoneName'])
    dfs = pool.imap_unordered(get_ground_truth, gr)
    dfs = tqdm(dfs, total=len(gr))
    dfs = list(dfs)
train_df = pd.concat(dfs).sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch'])
calc_haversine(train_df['latDeg'],train_df['lngDeg'], train_df['target_latDeg'], train_df['target_lngDeg']).mean()

  0%|          | 0/73 [00:00<?, ?it/s]

3.8468483749952074

In [11]:
train_df = linear_interpolation(train_df, speed_thr=45)
calc_haversine(train_df['latDeg'], train_df['lngDeg'], train_df['target_latDeg'], train_df['target_lngDeg']).mean()

3.6010082912730264

In [12]:
train_df = apply_kf_smoothing(train_df)
calc_haversine(train_df['latDeg'],train_df['lngDeg'], train_df['target_latDeg'], train_df['target_lngDeg']).mean()

  0%|          | 0/73 [00:00<?, ?it/s]

3.152398745382418

In [13]:
# data womizumasi
def make_lerp_data(df):
    '''
    Generate interpolated lat,lng values for different phone times in the same collection.
    '''
    org_columns = df.columns
    
    # Generate a combination of time x collection x phone and combine it with the original data (generate records to be interpolated)
    time_list = df[['collectionName', 'millisSinceGpsEpoch']].drop_duplicates()
    phone_list =df[['collectionName', 'phoneName']].drop_duplicates()
    tmp = time_list.merge(phone_list, on='collectionName', how='outer')
 
    lerp_df = tmp.merge(df, on=['collectionName', 'millisSinceGpsEpoch', 'phoneName'], how='left')

    lerp_df['phone'] = lerp_df['collectionName'] + '_' + lerp_df['phoneName']
    lerp_df = lerp_df.sort_values(['phone', 'millisSinceGpsEpoch'])

    # linear interpolation
    lerp_df['latDeg_prev'] = lerp_df['latDeg'].shift(1)
    lerp_df['latDeg_next'] = lerp_df['latDeg'].shift(-1)
    lerp_df['lngDeg_prev'] = lerp_df['lngDeg'].shift(1)
    lerp_df['lngDeg_next'] = lerp_df['lngDeg'].shift(-1)
    lerp_df['phone_prev'] = lerp_df['phone'].shift(1)
    lerp_df['phone_next'] = lerp_df['phone'].shift(-1)
    lerp_df['time_prev'] = lerp_df['millisSinceGpsEpoch'].shift(1)
    lerp_df['time_next'] = lerp_df['millisSinceGpsEpoch'].shift(-1)


    # Leave only records to be interpolated(missing coords data)
    lerp_df = lerp_df[(lerp_df['latDeg'].isnull())&(lerp_df['phone']==lerp_df['phone_prev'])&(lerp_df['phone']==lerp_df['phone_next'])].copy()
    # calc lerp
    lerp_df['latDeg'] = lerp_df['latDeg_prev'] + ((lerp_df['latDeg_next'] - lerp_df['latDeg_prev']) * ((lerp_df['millisSinceGpsEpoch'] - lerp_df['time_prev']) / (lerp_df['time_next'] - lerp_df['time_prev']))) 
    lerp_df['lngDeg'] = lerp_df['lngDeg_prev'] + ((lerp_df['lngDeg_next'] - lerp_df['lngDeg_prev']) * ((lerp_df['millisSinceGpsEpoch'] - lerp_df['time_prev']) / (lerp_df['time_next'] - lerp_df['time_prev']))) 
    
    # Leave only the data that has a complete set of previous and next data.
    lerp_df = lerp_df[~lerp_df['latDeg'].isnull()]

    return lerp_df[org_columns]


def calc_mean_pred(df, lerp_df):
    '''
    Make a prediction based on the average of the predictions of phones in the same collection.
    '''
    add_lerp = pd.concat([df, lerp_df])
    mean_pred_result = add_lerp.groupby(['collectionName', 'millisSinceGpsEpoch'])[['latDeg', 'lngDeg']].mean().reset_index()
    mean_pred_df = df[['collectionName', 'phoneName', 'millisSinceGpsEpoch']].copy()
    mean_pred_df = mean_pred_df.merge(mean_pred_result[['collectionName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['collectionName', 'millisSinceGpsEpoch'], how='left')
    return mean_pred_df

In [14]:
train_lerp = make_lerp_data(train_df)
train_df = calc_mean_pred(train_df, train_lerp)

In [15]:
import multiprocessing

processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    gr = train_df.groupby(['collectionName','phoneName'])
    dfs = pool.imap_unordered(get_ground_truth, gr)
    dfs = tqdm(dfs, total=len(gr))
    dfs = list(dfs)
train_df = pd.concat(dfs).sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch']).reset_index(drop=True)
calc_haversine(train_df['latDeg'],train_df['lngDeg'], train_df['target_latDeg'], train_df['target_lngDeg']).mean()

  0%|          | 0/73 [00:00<?, ?it/s]

2.9059163056773833

In [22]:
start_stop_epoch_list = []
end_stop_epoch_list = []
for (collection_name, phone_name), df in train_df.groupby(['collectionName', 'phoneName']):
    buffer = 60000
    min_epoch = df['millisSinceGpsEpoch'].min()
    max_epoch = df['millisSinceGpsEpoch'].max()
    start_stop_epoch_count = df.loc[df['millisSinceGpsEpoch'] < min_epoch+buffer, 'target_latDeg'].value_counts().to_numpy()[0]
    start_stop_epoch_list.append(start_stop_epoch_count)

    end_stop_epoch_count = df.loc[max_epoch-buffer < df['millisSinceGpsEpoch'], 'target_latDeg'].value_counts().to_numpy()[0]
    end_stop_epoch_list.append(start_stop_epoch_count)

In [49]:
_train_df = pd.DataFrame()
for (collection_name, phone_name), df in train_df.groupby(['collectionName', 'phoneName']):

    df = df.sort_values('millisSinceGpsEpoch').reset_index(drop=True)
    min_epoch = df['millisSinceGpsEpoch'].min()

    buffer = 60000  # 1min
    max_same_point_count = df.loc[df['millisSinceGpsEpoch'] < min_epoch+buffer, 'target_latDeg'].value_counts().to_numpy()[0]
    max_same_point = df.loc[df['millisSinceGpsEpoch'] < min_epoch+buffer, 'target_latDeg'].value_counts().index[0]

    # if stopping more 5s 
    if 10 < max_same_point:
        df.loc[:max_same_point_count, 'latDeg'] = df.loc[:max_same_point_count, 'latDeg'].mean()
        df.loc[:max_same_point_count, 'lngDeg'] = df.loc[:max_same_point_count, 'lngDeg'].mean()
    
    _train_df = pd.concat([_train_df, df])
_train_df = _train_df.sort_index()
calc_haversine(_train_df['latDeg'],_train_df['lngDeg'], _train_df['target_latDeg'], _train_df['target_lngDeg']).mean()

2.8956891315856725

In [18]:
def stopping_mean(input_df, buffer=10000):
    output_df = pd.DataFrame()
    for (collection_name, phone_name), df in input_df.groupby(['collectionName', 'phoneName']):
        df['']
        min_epoch = df['millisSinceGpsEpoch'].min()
        max_epoch = df['millisSinceGpsEpoch'].max()

        start_stopping_group_df = df[df['millisSinceGpsEpoch'] <= min_epoch + buffer]
        start_latdeg = start_stopping_group_df['latDeg'].median()
        start_lngdeg = start_stopping_group_df['lngDeg'].median()
        df.loc[df['millisSinceGpsEpoch'] <= min_epoch + buffer, 'latDeg'] = start_latdeg 
        df.loc[df['millisSinceGpsEpoch'] <= min_epoch + buffer, 'lngDeg'] = start_lngdeg 

        end_stopping_group_df = df[max_epoch-buffer <= df['millisSinceGpsEpoch']]
        end_latdeg = end_stopping_group_df['latDeg'].median()
        end_lngdeg = end_stopping_group_df['lngDeg'].median()
        df.loc[max_epoch - buffer <= df['millisSinceGpsEpoch'], 'latDeg'] = end_latdeg 
        df.loc[max_epoch - buffer <= df['millisSinceGpsEpoch'], 'lngDeg'] = end_lngdeg
        output_df = pd.concat([output_df, df])
    return output_df.sort_index()

In [24]:
_train_df = stopping_mean(train_df, buffer=3000)
calc_haversine(_train_df['latDeg'],_train_df['lngDeg'], _train_df['target_latDeg'], _train_df['target_lngDeg']).mean()

2.9095022263521746

In [25]:
import pickle
def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)
        
def from_pickle(filename):
    with open(filename, mode='rb') as f:
        obj = pickle.load(f)
    return obj

## test

In [41]:
test_df = pd.read_csv(root_dir / "baseline_locations_test.csv")
sub_df = pd.read_csv(root_dir / 'sample_submission.csv')

In [42]:
test_df = linear_interpolation(test_df)
test_df = apply_kf_smoothing(test_df)
test_lerp = make_lerp_data(test_df)
test_df = calc_mean_pred(test_df, test_lerp)
test_df = stopping_mean(test_df, buffer=30000)

  0%|          | 0/48 [00:00<?, ?it/s]

In [43]:
test_df['phone'] = test_df['collectionName'] + "_" + test_df['phoneName']
test_df

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,phone
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416603,-122.082064,2020-05-15-US-MTV-1_Pixel4
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416603,-122.082064,2020-05-15-US-MTV-1_Pixel4
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416603,-122.082064,2020-05-15-US-MTV-1_Pixel4
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416603,-122.082064,2020-05-15-US-MTV-1_Pixel4
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416603,-122.082064,2020-05-15-US-MTV-1_Pixel4
...,...,...,...,...,...,...
91481,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763185000,37.334548,-121.899406,2021-04-29-US-SJC-3_SamsungS20Ultra
91482,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763186000,37.334548,-121.899406,2021-04-29-US-SJC-3_SamsungS20Ultra
91483,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763187000,37.334548,-121.899406,2021-04-29-US-SJC-3_SamsungS20Ultra
91484,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763188000,37.334548,-121.899406,2021-04-29-US-SJC-3_SamsungS20Ultra


In [44]:
import plotly.express as px
def visualize_collection_for_sub(df, root_dir, collection_name='2021-03-16-US-MTV-2'):

    target_df = df[df['collectionName']==collection_name]   
    target_df['elapsed_epoch'] = ((target_df['millisSinceGpsEpoch'] - target_df['millisSinceGpsEpoch'].min()) / 1000).astype(str).to_numpy()
    fig = px.scatter_mapbox(target_df,
                                
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            text='elapsed_epoch',
                                
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                                
                            zoom=9,
                            center={"lat":37.423576, "lon":-122.094132},
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text=f"{collection_name}")
    fig.show()

In [45]:
visualize_collection_for_sub(test_df, root_dir)

<ipython-input-44-0ff0b9b93606>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
sub_df = sub_df.drop(['latDeg', 'lngDeg'], axis=1).merge(test_df[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'])
sub_df.to_csv('submission.csv', index=False)

In [ ]:
_sub_df = pd.read_csv('../exp009/submission.csv')
calc_haversine(sub_df['latDeg'],sub_df['lngDeg'],
               _sub_df['latDeg'], _sub_df['lngDeg']).mean()